In [1]:
import pandas as pd
from os.path import join, dirname, basename
import os
import numpy as np
from icecream import ic

#import seaborn as sns

import seaborn as sns

import matplotlib.pyplot as plt

In [3]:
allExpmts_df = snakemake.params.allExpmts 
allRuns_don_df = snakemake.params.allRuns_don_df
out_f = snakemake.output.cov #/data/Mito_Trace/output/aggregate/CHIP_aggr/results/clones/variants.init_knnRes.30/aggregate/clone_sizes.pdf
outdir = snakemake.params.outdir
#clone_method_dir = snakemake.params.clone_method_dir
#variants = snakemake.wildcards.variants
#kparam = snakemake.wildcards.kparam


#"/mnt/md0/isshamie/Projects/Mito_Trace/output/pipeline/v02/CHIP_b1/MTBlacklist_A2/data/merged/MT/cellr_True/numread_200/filters/minC10_minR50_topN0_hetT0.001_hetC10_hetCount5_bq20/mgatk/vireoIn/multiplex/clones_init/donor0/dp.tsv"
# kparam = 30
# variants = "init"
# output = ""
#
# clone_dirs = []
# nuclear_dirs = []
# condition_pileup_dirs = []
# experiment_ids = []

In [ ]:
# clone_method_dir = join("clones/", f"variants_{variants}/knn/kparam_{kparam}")
# clone_method_dir

## Get number of cells each condition

In [4]:
#init_results_cond_df = pd.DataFrame(columns=["Donor", "# conditions"])
results_donor_df = allRuns_don_df.copy()
results_donor_df["# cells after demultiplex"] = -1

results_af_d = []# {}
results_cov_d = []
## Get multiplex ncells
for ind, val in allExpmts_df.groupby("run"):
    print(ind)
    for ind2, val2 in val.iterrows():
        #mgatk/Flt3l.af.tsv mgatk/Flt3l.coverage.txt
        if not os.path.exists(join(val2["filt_dir"],f"mgatk/{val2['sample_name']}.af.tsv")):
            print("Not here", ind2)
        else:
            curr = pd.read_csv(join(val2["filt_dir"],f"mgatk/{val2['sample_name']}.af.tsv"), sep="\t", index_col=0).transpose()
            curr.index = [f"{x}_{ind2}" for x in curr.index]
            curr = curr.reset_index().melt(id_vars='index')
            curr["condition"] = val2['sample_name']
            curr["run"] = val2['run']
            curr["ID"] = ind2
            results_af_d.append(curr)
            curr = pd.read_csv(join(val2["filt_dir"],f"mgatk/{val2['sample_name']}.coverage.tsv"), sep="\t", index_col=0).transpose()
            curr.index = [f"{x}_{ind2}" for x in curr.index]
            curr = curr.reset_index().melt(id_vars='index')
            curr["condition"] = val2['sample_name']
            curr["run"] = val2['run']
            curr["ID"] = ind2
            results_cov_d.append(curr)
    
results_af_df = pd.concat(results_af_d).fillna(0)
results_af_df

#"mean clone size", "median clone size"

results_cov_df = pd.concat(results_cov_d).fillna(0)
results_cov_df

In [11]:
sns.set_context(font_scale=1.2)
results_cov_df["log2_count"] = np.log2(1+results_cov_df["value"])
g = sns.catplot(data=results_cov_df, x="run", hue="condition", height=6, 
                kind="violin", y="log2_count")
g.fig.savefig(fname=join(outdir, f"coverage_log2_oneRow.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)




In [6]:
results_cov_df["log2_count"] = np.log2(1+results_cov_df["value"])
g = sns.catplot(data=results_cov_df.sample(1000), row="run", col="condition", height=4, 
                kind="violin", y="log2_count", inner="stick")

g.fig.savefig(fname=join(outdir, f"coverage_log2.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

#g.map_dataframe(sns.violinplot, y="value", hue="condition" )

In [7]:
## Violinplot function
def plot_violin(data, **kwargs):
    #print(data.columns)
    curr_type = kwargs.get("method", "median")
    curr_axis = kwargs.get("axis", 0)
    curr_dat = (data.pivot(index="index", columns="variable", values="value").fillna(0))
    if curr_type == "median":
        sns.violinplot(x=curr_dat.median(axis=curr_axis))
    elif curr_type == "mean":
        sns.violinplot(x=curr_dat.mean(axis=curr_axis))
    return



## Plot median+mean cell coverage for each position

In [8]:
g = sns.FacetGrid(data=results_cov_df, row="run", col="condition", height=4)
g.map_dataframe(plot_violin, method="median")
g.fig.suptitle("median coverage for each called variant genomic loci")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"coverage_median_per_variant.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

g = sns.FacetGrid(data=results_cov_df, row="run", col="condition", height=4)
g.map_dataframe(plot_violin, method="mean")
g.fig.suptitle("mean coverage for each called variant genomic loci")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"coverage_mean_per_variant.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

## Plot median+cell position coverage for each cell

In [9]:
g = sns.FacetGrid(data=results_cov_df, row="run", col="condition")
g.map_dataframe(plot_violin, method="median", axis=1)
g.fig.suptitle("median coverage for each cell over the MT genomic positions")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"coverage_median_per_cell.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

g = sns.FacetGrid(data=results_cov_df, row="run", col="condition")
g.map_dataframe(plot_violin, method="mean", axis=1)
g.fig.suptitle("mean coverage for each cell over the MT genomic positions")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"coverage_mean_per_cell.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

## Plot median+mean cell AF for each position

In [10]:
g = sns.FacetGrid(data=results_af_df, row="run", col="condition")
g.map_dataframe(plot_violin, method="median")
g.fig.suptitle("median coverage for each called variant genomic loci")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"af_median_per_variant.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

g = sns.FacetGrid(data=results_af_df, row="run", col="condition")
g.map_dataframe(plot_violin, method="mean")
g.fig.suptitle("mean coverage for each called variant genomic loci")
plt.tight_layout()
g.fig.savefig(fname=join(outdir, f"af_mean_per_variant.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

## Calculate barplot coverage plot

In [11]:
def plot_percent_coverage_cells(data,
                                x_cov=(5, 10, 30, 50, 100, 200, 300),
                                cells_cov = (1, 10, 100, 500, 1000,5000,10000), **kwargs):
    # x_cov = [1, 2, 5, 10, 30, 50, 100, 200, 1000, 2000, 3000]
    # cells_cov = [1, 10, 100, 500]

    pct_cov_df = pd.DataFrame(
        columns=["Number of MT positions", "Minimum number of cells",
                 "Minimum number of reads"])

    for x in x_cov:
        sc_coverage = data.pivot(index="index", columns="variable", values="value").fillna(0)
        #curr_count = (sc_coverage >= x).sum(axis=0)
        curr_count = (sc_coverage >= x).sum(axis=1)
        
        for c in cells_cov:
            pct_cov_df = pd.concat((pct_cov_df, pd.DataFrame(
                {"Number of MT positions": (curr_count >= c).sum(),
                 "Minimum number of cells": c,
                 "Minimum number of reads": x}, index=[len(
                    pct_cov_df)])))  # pct_cov.append((sc_coverage > x).sum(axis=0).sum())
    sns.barplot(data=pct_cov_df, x="Minimum number of reads",
                y="Number of MT positions",
                hue="Minimum number of cells", ax=plt.gca())
    plt.xticks(rotation=45)
    return

g = sns.FacetGrid(data=results_cov_df, row="run", col="condition", height=4)
g.map_dataframe(plot_percent_coverage_cells)
plt.legend(loc='best')
g.fig.savefig(fname=join(outdir, f"coverage_ncells_npositions.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

In [12]:
g = sns.FacetGrid(data=results_cov_df, row="run", col="condition", sharey=False)
g.map_dataframe(plot_percent_coverage_cells)
plt.legend(loc='best')
g.fig.savefig(fname=join(outdir, f"coverage_ncells_npositions.diff_y_axis.pdf"), dpi=500, bbox_inches='tight', pad_inches=0.5)

In [13]:
#results_df.to_csv(out_f, sep="\t")

### Plot average across genome (make circular?)

In [14]:
results_cov_df["position"] = results_cov_df["variable"].apply(lambda x: x.split(">")[0][:-1]).astype(int)

pos_mean = results_cov_df.groupby(["position","ID"]).mean().sort_index()
pos_mean["log2_count"] = np.log2(1+pos_mean[["value"]])

g=sns.FacetGrid(pos_mean.reset_index(), col="ID", col_wrap=2 )
g.map_dataframe(sns.regplot,x="position", y="log2_count")
plt.savefig(join(outdir, "position_mean.svg"))
plt.savefig(join(outdir, "position_mean.pdf"))

g=sns.FacetGrid(pos_mean.reset_index(), col="ID", col_wrap=2 , sharey=False)
g.map_dataframe(sns.regplot,x="position", y="log2_count")
plt.savefig(join(outdir, "position_mean_diffY.svg"))
plt.savefig(join(outdir, "position_mean_diffY.pdf"))



In [15]:
# upperLimit = 100
# lowerLimit = 30

# # Compute max and min in the dataset
# max = df['Value'].max()

# # Let's compute heights: they are a conversion of each item value in those new coordinates
# # In our example, 0 in the dataset will be converted to the lowerLimit (10)
# # The maximum will be converted to the upperLimit (100)
# slope = (max - lowerLimit) / max
# heights = slope * df.Value + lowerLimit

# # Compute the width of each bar. In total we have 2*Pi = 360°
# width = 2*np.pi / len(df.index)

# # Compute the angle each bar is centered on:
# indexes = list(range(1, len(df.index)+1))
# angles = [element * width for element in indexes]
# angles

# # Draw bars
# bars = ax.bar(
#     x=angles, 
#     height=heights, 
#     width=width, 
#     bottom=lowerLimit,
#     linewidth=2, 
#     edgecolor="white")